# --- Settings ---

In [ ]:
# Standard library
import os, time, json, csv, multiprocessing as mp
from time import perf_counter

# Third-party
import numpy as np, pandas as pd, networkx as nx, osmnx as ox, umap
import matplotlib as mpl, matplotlib.pyplot as plt
from node2vec import Node2Vec
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from PIL import Image, ImageDraw, ImageFont

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
warnings.filterwarnings("ignore", category=UserWarning, module="umap")
warnings.filterwarnings("ignore", category=UserWarning, module="joblib.externals.loky")

# Fast BLAS on Apple Silicon
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

# OSMnx settings
ox.settings.use_cache = True
ox.settings.log_console = False

# Files & directories
pdf_file = "comparison.pdf"
map_dir, tax_dir, emb_dir, gra_dir, clu_dir = "maps", "taxonomy", "embeddings", "graphs", "clusters"
for d in (map_dir, tax_dir, emb_dir, gra_dir, clu_dir):
    os.makedirs(d, exist_ok=True)

# ---- GSD-inspired earthy mapping palette ----
NEUTRAL = "#CCCCCC"
PALETTE = [
    "#A4653E",  # brighter terracotta
    "#D9985E",  # lighter ochre
    "#88B28B",  # fresher sage green
    "#69A8B2",  # clearer teal
    "#B2879C",  # livelier mauve
    "#6A717A",  # lighter slate gray
    "#E0D3B0",  # brighter pale stone
    "#9C7384",  # stronger plum
]

city_data = [
    {"name": "Rome", "country": "ITA",
        "coordinates": (41.894096, 12.485609), "distance": 12000, "network": "drive",
        "group": "Archetypal", "taxonomy": "Radial_Implosion"},
    {"name": "Vatican_City", "country": "VAT",
        "coordinates": (41.902257, 12.457421), "distance": 200, "network": "all",
        "group": "Archetypal", "taxonomy": "Elliptical_Implosion"},
    {"name": "Fez", "country": "MAR",
        "coordinates": (34.065, -4.973), "distance": 800, "network": "all",
        "group": "Archetypal", "taxonomy": "Organic_Rhizome"},
    {"name": "Moscow", "country": "RUS",
        "coordinates": (55.7558, 37.6176), "distance": 60000, "network": "drive",
        "group": "Archetypal", "taxonomy": "Centralized_Burst"},
    {"name": "Medellin", "country": "COL",
        "coordinates": (6.2518, -75.5836), "distance": 15000, "network": "all",
        "group": "Geometrical", "taxonomy": "Arc_Diagram"},
    {"name": "Palmanova", "country": "ITA",
        "coordinates": (45.9061, 13.3095), "distance": 1500, "network": "all",
        "group": "Geometrical", "taxonomy": "Radial_Convergence"},
    {"name": "Dubai", "country": "ARE",
        "coordinates": (25.056530, 55.207939), "distance": 1000, "network": "all",
        "group": "Geometrical", "taxonomy": "Segmented_Radial_Convergence"},
    {"name": "Canberra", "country": "AUS",
        "coordinates": (-35.308188, 149.124441), "distance": 3200, "network": "all",
        "group": "Geometrical", "taxonomy": "Centralized_Ring"},
    {"name": "Los_Angeles", "country": "USA",
        "coordinates": (34.029315, -118.214444), "distance": 800, "network": "drive",
        "group": "Relational", "taxonomy": "Flow_Chart"},
    {"name": "Randstad", "country": "NLD",
        "coordinates": (52.1, 4.6), "distance": 40000, "network": "drive",
        "group": "Relational", "taxonomy": "Area_Grouping"},
    {"name": "Greater_Cairo", "country": "EGY",
        "coordinates": (30.0444, 31.2357), "distance": 50000, "network": "drive",
        "group": "Relational", "taxonomy": "Circular_Ties"},
    {"name": "Amsterdam", "country": "NLD",
        "coordinates": (52.371, 4.90), "distance": 2000, "network": "all",
        "group": "Relational", "taxonomy": "Ramification"}
]

# --- Choose which cities to process ---
TARGET_CITIES = None   # None = process all cities
# TARGET_CITIES = ["Amsterdam", "Canberra"]  # uncomment to process only Amsterdam

print(f"{len(city_data)} cities loaded")

12 cities loaded


# --- Graphs ---

In [2]:
for city in city_data:
    if TARGET_CITIES and city['name'] not in TARGET_CITIES:
        continue
    
    gpath = os.path.join(gra_dir, f"{city['name']}.graphml")

    # if os.path.exists(gpath):
    #     print(f"🗂️ {city['name']} skipped (already exists)")
    #     continue

    t0 = perf_counter()
    G = ox.graph_from_point(
        city['coordinates'],
        dist=city['distance'],
        network_type=city['network'],
        simplify=True,
        retain_all=False
    )
    ox.save_graphml(G, gpath)
    dt = perf_counter() - t0
    print(f"✅ {city['name']} ({city['network']}, r={city['distance']}m) — saved in {dt:.1f}s")

✅ Rome (drive, r=12000m) — saved in 43.3s
✅ Vatican_City (all, r=200m) — saved in 0.9s
✅ Fez (all, r=800m) — saved in 2.0s
✅ Moscow (drive, r=60000m) — saved in 128.6s
✅ Medellin (all, r=15000m) — saved in 62.9s
✅ Palmanova (all, r=1500m) — saved in 0.9s
✅ Dubai (all, r=1000m) — saved in 2.3s
✅ Canberra (all, r=3200m) — saved in 12.8s
✅ Los_Angeles (drive, r=800m) — saved in 0.4s
✅ Randstad (drive, r=40000m) — saved in 189.8s
✅ Greater_Cairo (drive, r=50000m) — saved in 332.0s
✅ Amsterdam (all, r=2000m) — saved in 8.1s


# --- Node2Vec ---

In [3]:
for city in city_data:
    if TARGET_CITIES and city['name'] not in TARGET_CITIES:
        continue

    npz_path = os.path.join(emb_dir, f"{city['name']}.npz")
    ids_path = os.path.join(emb_dir, f"{city['name']}.ids")

    t0 = perf_counter()

    # largest connected component → a single “story world” for the walks
    H = nx.Graph(ox.load_graphml(os.path.join(gra_dir, f"{city['name']}.graphml")))
    H = H.subgraph(max(nx.connected_components(H), key=len)).copy()
    node_list = list(H.nodes())

    # Node2Vec sampler (Grover & Leskovec): biased 2nd-order random walks
    n2v = Node2Vec(
        H,
        dimensions=32,          # embedding dimensionality (the space the walks learn)
        walk_length=15,         # T: length of each 2nd-order walk
        num_walks=8,            # γ: walks launched per node (more = smoother estimates)
        p=1.0,                  # return parameter: >1 discourages immediate backtracking; <1 encourages it
        q=0.5,                  # in/out parameter: <1 → BFS/homophily (stay local); >1 → DFS/structural roles
        workers=max(1, mp.cpu_count()-1),  # parallel samplers
        seed=42,                # reproducible walk corpus
        quiet=True              # hush the sampler
    )

    # Skip-gram optimizer on the walk corpus (word2vec on nodes)
    model = n2v.fit(
        window=5,               # context window (how far co-occurrence is trusted)
        min_count=1,            # keep all nodes in the vocabulary
        batch_words=128         # SGD batch size
    )

    # embeddings aligned to node_list order
    X = np.array([model.wv[str(n)] for n in node_list])

    np.savez_compressed(npz_path, X=X)
    with open(ids_path, "w") as f:
        f.write("\n".join(map(str, node_list)))

    print(f"   ✅ Saved {city['name']} ({len(node_list)} nodes) in {perf_counter()-t0:.1f}s")

   ✅ Saved Rome (32269 nodes) in 32.5s
   ✅ Saved Vatican_City (149 nodes) in 0.2s
   ✅ Saved Fez (1506 nodes) in 1.1s
   ✅ Saved Moscow (107428 nodes) in 102.7s
   ✅ Saved Medellin (69842 nodes) in 72.7s
   ✅ Saved Palmanova (678 nodes) in 0.8s
   ✅ Saved Dubai (1763 nodes) in 1.3s
   ✅ Saved Canberra (14523 nodes) in 13.1s
   ✅ Saved Los_Angeles (184 nodes) in 0.2s
   ✅ Saved Randstad (183497 nodes) in 191.7s
   ✅ Saved Greater_Cairo (466594 nodes) in 584.5s
   ✅ Saved Amsterdam (10087 nodes) in 13.2s


# --- UMAP ---

In [4]:
for city in city_data:
    if TARGET_CITIES is not None and city['name'] not in TARGET_CITIES:
        continue

    t0 = perf_counter()
    X = np.load(os.path.join(emb_dir, f"{city['name']}.npz"))["X"].astype("float32")
    umap_path = os.path.join(emb_dir, f"{city['name']}.npy")

    # Normalize + PCA
    X_red = PCA(n_components=16, random_state=0).fit_transform(normalize(X, norm="l2"))
    t_pca = perf_counter()

    # UMAP
    embed = umap.UMAP(
        n_neighbors=8,     # 8 words nicely to keep the global structure (5 leaves holes in the structure of the city)
        min_dist=0.20,      # was 0.20; gives a nice balance between local and global structure
        metric="euclidean", # fine on normalized/PCA data
        random_state=42,
        n_epochs=150,
        low_memory=True,
    ).fit_transform(X_red)

    np.save(umap_path, embed)
    print(f"🏙️ {city['name']} | PCA {t_pca-t0:.1f}s | UMAP {perf_counter()-t_pca:.1f}s | saved {umap_path}")

🏙️ Rome | PCA 0.0s | UMAP 17.6s | saved embeddings/Rome.npy
🏙️ Vatican_City | PCA 0.0s | UMAP 2.0s | saved embeddings/Vatican_City.npy
🏙️ Fez | PCA 0.0s | UMAP 1.1s | saved embeddings/Fez.npy
🏙️ Moscow | PCA 0.1s | UMAP 33.1s | saved embeddings/Moscow.npy
🏙️ Medellin | PCA 0.0s | UMAP 20.7s | saved embeddings/Medellin.npy
🏙️ Palmanova | PCA 0.0s | UMAP 0.3s | saved embeddings/Palmanova.npy
🏙️ Dubai | PCA 0.0s | UMAP 1.4s | saved embeddings/Dubai.npy
🏙️ Canberra | PCA 0.0s | UMAP 4.1s | saved embeddings/Canberra.npy
🏙️ Los_Angeles | PCA 0.0s | UMAP 0.0s | saved embeddings/Los_Angeles.npy
🏙️ Randstad | PCA 0.1s | UMAP 60.2s | saved embeddings/Randstad.npy
🏙️ Greater_Cairo | PCA 0.3s | UMAP 192.3s | saved embeddings/Greater_Cairo.npy
🏙️ Amsterdam | PCA 0.0s | UMAP 3.1s | saved embeddings/Amsterdam.npy


# --- HDBSCAN ---

In [5]:
for city in city_data:
    if TARGET_CITIES and city['name'] not in TARGET_CITIES:
        continue

    t0 = perf_counter()

    # Load UMAP embedding
    umap_path = os.path.join(emb_dir, f"{city['name']}.npy")
    embed = np.load(umap_path)

    # Load node ids; fallback to sequential ids if file is missing
    ids_path = os.path.join(emb_dir, f"{city['name']}.ids")
    with open(ids_path) as f:
        node_list = [line.strip() for line in f]

    # Sanity check: rows must match ids count
    assert embed.shape[0] == len(node_list), (
        f"Mismatch: {embed.shape[0]} embeddings vs {len(node_list)} ids for {city['name']}"
    )

    # HDBSCAN on UMAP space (2D)
    clusterer = HDBSCAN(
        min_cluster_size=max(10, int(len(node_list) ** 0.70)),
        min_samples=3,
        metric="euclidean",
        cluster_selection_method="leaf",
        cluster_selection_epsilon=0.06,
        prediction_data=False,
        approx_min_span_tree=True,
        gen_min_span_tree=False,
        algorithm="best",
        core_dist_n_jobs=mp.cpu_count(),
    )
    labels = clusterer.fit_predict(embed)  # -1 = noise

    # Map cluster → color using the GLOBAL PALETTE (stable order)
    uniq = sorted(set(labels) - {-1})
    label2color = {lab: PALETTE[i % len(PALETTE)] for i, lab in enumerate(uniq)}
    point_colors = [label2color.get(lbl, NEUTRAL) for lbl in labels]

    # Save colored embedding (for visual checks)
    out_jpg = os.path.join(clu_dir, f"{city['name']}.jpg")
    plt.figure(figsize=(8, 8))
    plt.scatter(embed[:, 0], embed[:, 1], s=1.5, c=point_colors, alpha=0.9, edgecolor='none')
    plt.axis("off")
    plt.savefig(out_jpg, dpi=600, bbox_inches="tight", format="jpg")
    plt.close()

    # Persist exact colors per node
    csv_path = os.path.join(clu_dir, f"{city['name']}.csv")
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["node_id", "cluster", "color_hex"])
        for nid, lbl, col in zip(node_list, labels, point_colors):
            w.writerow([nid, int(lbl), col])

    noise = int((labels == -1).sum())
    print(f"🏙️ {city['name']} | HDBSCAN {perf_counter()-t0:.1f}s | clusters {len(uniq)} | noise {noise}/{len(node_list)} ({noise/len(node_list):.1%}) → {csv_path}")

🏙️ Rome | HDBSCAN 2.2s | clusters 7 | noise 11376/32269 (35.3%) → clusters/Rome.csv
🏙️ Vatican_City | HDBSCAN 0.1s | clusters 3 | noise 42/149 (28.2%) → clusters/Vatican_City.csv
🏙️ Fez | HDBSCAN 0.1s | clusters 4 | noise 578/1506 (38.4%) → clusters/Fez.csv
🏙️ Moscow | HDBSCAN 2.0s | clusters 5 | noise 77533/107428 (72.2%) → clusters/Moscow.csv
🏙️ Medellin | HDBSCAN 1.3s | clusters 5 | noise 44829/69842 (64.2%) → clusters/Medellin.csv
🏙️ Palmanova | HDBSCAN 0.1s | clusters 3 | noise 91/678 (13.4%) → clusters/Palmanova.csv
🏙️ Dubai | HDBSCAN 0.1s | clusters 3 | noise 342/1763 (19.4%) → clusters/Dubai.csv
🏙️ Canberra | HDBSCAN 0.3s | clusters 7 | noise 3553/14523 (24.5%) → clusters/Canberra.csv
🏙️ Los_Angeles | HDBSCAN 0.1s | clusters 3 | noise 10/184 (5.4%) → clusters/Los_Angeles.csv
🏙️ Randstad | HDBSCAN 3.6s | clusters 0 | noise 183497/183497 (100.0%) → clusters/Randstad.csv
🏙️ Greater_Cairo | HDBSCAN 11.3s | clusters 2 | noise 309779/466594 (66.4%) → clusters/Greater_Cairo.csv
🏙️ Ams

# --- Maps ---

In [6]:
for city in city_data:
    if TARGET_CITIES and city['name'] not in TARGET_CITIES:
        continue

    t0 = perf_counter()

    # Load graph
    G = ox.load_graphml(os.path.join(gra_dir, f"{city['name']}.graphml"))

    # Load cluster labels and the exact colors chosen earlier
    node_cluster, node_color = {}, {}
    csv_path = os.path.join(clu_dir, f"{city['name']}.csv")
    with open(csv_path, newline="") as f:
        r = csv.DictReader(f)
        for row in r:
            nid = str(row["node_id"])
            node_cluster[nid] = int(row["cluster"])
            node_color[nid]   = row.get("color_hex") or NEUTRAL

    # Project and color edges: use a cluster color only when both endpoints share the same cluster; else NEUTRAL
    G_proj = ox.project_graph(G)
    edge_colors = [
        (node_color.get(str(u), NEUTRAL)
         if (node_cluster.get(str(u)) == node_cluster.get(str(v)) and
             node_cluster.get(str(u), -1) != -1)
         else NEUTRAL)
        for u, v, k in G_proj.edges(keys=True)
    ]

    out_png = os.path.join(map_dir, f"{city['name']}.png")
    fig, ax = ox.plot_graph(
        G_proj,
        bgcolor="white",
        node_size=0,
        edge_color=edge_colors,
        edge_linewidth=0.4,
        show=False,
        save=True,
        filepath=out_png,
        dpi=300,
    )
    plt.close(fig)

    print(f"🗺️ {city['name']} | map {perf_counter()-t0:.1f}s → {out_png}")

🗺️ Rome | map 9.7s → maps/Rome.png
🗺️ Vatican_City | map 0.2s → maps/Vatican_City.png
🗺️ Fez | map 1.1s → maps/Fez.png
🗺️ Moscow | map 33.0s → maps/Moscow.png
🗺️ Medellin | map 21.6s → maps/Medellin.png
🗺️ Palmanova | map 0.4s → maps/Palmanova.png
🗺️ Dubai | map 0.6s → maps/Dubai.png
🗺️ Canberra | map 5.7s → maps/Canberra.png
🗺️ Los_Angeles | map 0.3s → maps/Los_Angeles.png
🗺️ Randstad | map 54.5s → maps/Randstad.png
🗺️ Greater_Cairo | map 173.2s → maps/Greater_Cairo.png
🗺️ Amsterdam | map 5.8s → maps/Amsterdam.png


# --- Panels ---

In [ ]:
# Layout params
thumb_size = (1500, 1500)
FONT_SIZE  = 40                 # one font size for everything
margin     = 40
title_y    = 20                  # top padding for the title

# Try a few common fonts; fall back to PIL default if none found
font = None
for fp in ["arial.ttf", "Arial.ttf", "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
           "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
           "/Library/Fonts/Arial.ttf", "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"]:
    try:
        font = ImageFont.truetype(fp, FONT_SIZE)
        break
    except Exception:
        continue
if font is None:
    font = ImageFont.load_default()

def text_width(draw, text, font):
    if hasattr(draw, "textlength"):
        return draw.textlength(text, font=font)
    # Fallback: use bbox width
    bbox = draw.textbbox((0, 0), text, font=font)
    return bbox[2] - bbox[0]

slides = []

for city in city_data:
    city_path     = os.path.join(map_dir, f"{city['name']}.png")
    clusters_path = os.path.join(clu_dir, f"{city['name']}.jpg")

    city_img     = Image.open(city_path).convert("RGB").resize(thumb_size, Image.LANCZOS)
    clusters_img = Image.open(clusters_path).convert("RGB").resize(thumb_size, Image.LANCZOS)

    images = [city_img, clusters_img]

    # Panel size
    panel_width  = len(images) * thumb_size[0] + (len(images) + 1) * margin
    panel_height = thumb_size[1] + FONT_SIZE + 120  # room for big title
    panel = Image.new("RGB", (panel_width, panel_height), "white")
    draw = ImageDraw.Draw(panel)

    # Paste images
    y = title_y + FONT_SIZE + 40
    for i, img in enumerate(images):
        x = margin + i * (thumb_size[0] + margin)
        panel.paste(img, (x, y))

    # Title: name + taxonomy + coordinates + type + radius
    coords = f"({city['coordinates'][0]:.4f}, {city['coordinates'][1]:.4f})"
    title_text = f"{city['name']} — {city['taxonomy']} — {coords}  •  type={city['network']}, r={city['distance']} m"
    tw = text_width(draw, title_text, font)
    draw.text(((panel_width - tw) // 2, title_y), title_text, font=font, fill="black")

    slides.append(panel)

# Export to PDF (all slides)
comparison_images_rgb = [img.convert("RGB") for img in slides]
comparison_images_rgb[0].save(pdf_file, save_all=True, append_images=comparison_images_rgb[1:], format="PDF")
print(f"📄 Exported to: {pdf_file}")

📄 Exported to: comparison.pdf
